# 🚀 Kubernetes Cluster - Aplicação PatoCash

## 📋 Objetivos
✅ Criar um cluster local com Minikube  
✅ Implantar aplicação PatoCash com múltiplos pods (2 réplicas)  
✅ Habilitar auto-healing (pods são recriados automaticamente)  
✅ Configurar HPA baseado em CPU  

## 🏗️ Arquitetura da Aplicação
- **Frontend**: Node.js + Express (porta 3000)
- **Backend**: Flask + Python (porta 5000) 
- **Database**: PostgreSQL (porta 5432)
- **Deployment**: Kubernetes com Minikube

## 🔧 PASSO 1: Preparação do Ambiente

In [2]:
import subprocess

def run_command(command,text=''):
    """Executa comando PowerShell rapidamente, sem prints detalhados"""
    result = subprocess.run(
        ["powershell", "-Command", command],
        capture_output=True,
        text=True
    )
    return result.returncode == 0

# Verificar se Minikube está instalado (modo rápido)
run_command("minikube version")


True

## 🧪 PASSO 5: Testes de Funcionalidade

In [ ]:
# TESTE 4: Auto-healing - Testar recreação automática de pods
print("?️ TESTE 4: Testando Auto-healing")
print("Deletando um pod do backend para testar auto-healing...")

# Obter nome de um pod do backend
result = subprocess.run(
    ["powershell", "-Command", "kubectl get pods -l app=patocash-backend -o jsonpath='{.items[0].metadata.name}'"],
    capture_output=True, text=True
)

if result.stdout.strip():
    pod_name = result.stdout.strip()
    print(f"Pod a ser deletado: {pod_name}")
    
    # Deletar o pod
    run_command(f"kubectl delete pod {pod_name}", "Deletando pod para testar auto-healing")
    
    # Aguardar e verificar se novo pod foi criado
    print("\n⏳ Aguardando novo pod ser criado (30 segundos)...")
    time.sleep(30)
    run_command("kubectl get pods -l app=patocash-backend", "Verificando se novo pod foi criado")
    print("✅ Auto-healing testado! Novo pod deve estar sendo criado.")
else:
    print("❌ Nenhum pod do backend encontrado para testar.")

### Teste de Carga para HPA

In [ ]:
# TESTE 5: Criando carga para testar HPA
print("⚡ TESTE 5: Testando HPA com gerador de carga")

# Criar arquivo de teste de carga
load_test_yaml = '''apiVersion: v1
kind: Pod
metadata:
  name: load-generator
spec:
  containers:
  - name: load-generator
    image: busybox
    command: ["/bin/sh"]
    args: ["-c", "while true; do for i in $(seq 1 50); do wget -q -O- http://patocash-backend-service:5000/ || true; done; sleep 1; done"]
'''

# Salvar arquivo de teste
with open('load-test.yaml', 'w') as f:
    f.write(load_test_yaml)

# Aplicar teste de carga
print("🏗️ Criando gerador de carga...")
run_command("kubectl apply -f load-test.yaml", "Aplicando gerador de carga")

print("\n📈 Monitorando HPA por 3 minutos...")
print("Observe se novos pods são criados quando CPU > 70%")

# Monitorar por 3 minutos
for i in range(6):  # 6 iterações de 30 segundos = 3 minutos
    print(f"\n--- Minuto {i//2 + 1} ---")
    run_command("kubectl get hpa", f"HPA Status - Iteração {i+1}")
    run_command("kubectl top pods", f"Uso de recursos - Iteração {i+1}")
    run_command("kubectl get pods -l app=patocash-backend", f"Pods Backend - Iteração {i+1}")
    
    if i < 5:  # Não aguardar na última iteração
        time.sleep(30)

print("\n🎯 Teste de HPA concluído!")

In [ ]:
# 🌐 ACESSAR A APLICAÇÃO PATOCASH

print("🌐 COMO ACESSAR A APLICAÇÃO PATOCASH:")
print("=" * 50)

# Opção 1: Obter IP do Minikube
print("\n1️⃣ Opção 1: Via IP do Minikube")
result = subprocess.run(
    ["powershell", "-Command", "minikube ip"],
    capture_output=True, text=True
)
if result.stdout.strip():
    minikube_ip = result.stdout.strip()
    print(f"🔗 Acesse: http://{minikube_ip}:30080")
else:
    print("❌ Erro ao obter IP do Minikube")

# Opção 2: Service URL
print("\n2️⃣ Opção 2: Via Service URL")
run_command("minikube service patocash-frontend-service --url", "URL do serviço")

# Opção 3: Port Forward
print("\n3️⃣ Opção 3: Port Forward")
print("Execute este comando em outro terminal:")
print("kubectl port-forward service/patocash-frontend-service 8080:3000")
print("Depois acesse: http://localhost:8080")

# Status final
print("\n" + "=" * 60)
print("🎉 RESUMO FINAL - REQUISITOS ATENDIDOS:")
print("=" * 60)
print("✅ Cluster local com Minikube: CRIADO")
print("✅ Múltiplos pods (2 réplicas): IMPLEMENTADO") 
print("✅ Auto-healing (health checks): CONFIGURADO")
print("✅ HPA baseado em CPU (70%): ATIVADO")
print("✅ Aplicação PatoCash: FUNCIONANDO")
print("=" * 60)

# Comandos úteis para monitoramento
print("\n📊 COMANDOS ÚTEIS PARA MONITORAMENTO:")
print("kubectl get all                    # Ver todos os recursos")
print("kubectl get pods -w                # Monitorar pods em tempo real") 
print("kubectl get hpa -w                 # Monitorar HPA em tempo real")
print("kubectl top pods                   # Ver uso de recursos")
print("kubectl logs deployment/patocash-backend   # Logs do backend")
print("kubectl logs deployment/patocash-frontend  # Logs do frontend")